# AnthropicToolCallState

```lua
stateDiagram-v2
    INIT --> CHAT
    CHAT --> FINAL
```

```mermaid
stateDiagram-v2
    Direction LR
    INIT --> CHAT
    CHAT --> FINAL
```


### a) Normal Chat Flow

In [1]:
import os
os.environ["LOG_LEVEL"]="WARNING"

from gai.lib.config import config_helper
llm_config = config_helper.get_client_config("sonnet-4")

In [2]:
from gai.asm import AgenticStateMachine

with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> CHAT
    CHAT--> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                }
            },
            "CHAT": {
                "module_path": "gai.asm.agents.tool_use_agent",
                "class_name": "AnthropicChatState",
                "title": "CHAT",
                "input_data": {
                    "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                },
                "output_data": ["streamer", "get_assistant_message"],
            },
            "FINAL": {
                "output_data": ["monologue"],
            },
        },
        get_llm_config=lambda state: llm_config
    )
    fsm.restart()

## Step 2: INIT --> TOOL_CALL
fsm.user_message = "Tell me a one sentence story."
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    if (isinstance(chunk,str)):
        print(chunk, end='', flush=True)
print("\n\n")

## Step 4: Print the state history
for message in fsm.state_history[1]["output"]["monologue"].list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

The old lighthouse keeper discovered that the mysterious light old lighthouse keeper discovered that the mysterious light he'd been seeing each night wasn he'd been seeing each night wasn't from a ship, but from his't from a ship, but from his own lantern reflecting own lantern reflecting off the tears he she off the tears he shed for his long-lost loved for his long-lost love.



1754270273.545431 User > 
            Your name is Assistant within the context of this conversation and you will always respond as such.
            Do not refer to yourself as an AI or a bot or confuse your name with other agents.
           
            You may respond to my following message using the context you have learnt.
            Tell me a one sentence story.
            
            You may ask me for more information if you need to clarify my request but ask just enough to get the information you need to get started.
            If you need to ask for more information, please use the "user_input

### b) With Dialogue Recap

In [3]:
import os
from gai.messages import Dialogue, MessagePydantic

messages = [
    MessagePydantic(
        **{
            "id": "b1e5f98c-f6eb-47de-a6e2-387510d970f9",
            "header": {
                "sender": "User",
                "recipient": "Sara",
                "timestamp": 1751308157.270983,
                "order": 0,
            },
            "body": {
                "type": "chat.send",
                "dialogue_id": "00000000-0000-0000-0000-000000000000",
                "round_no": 0,
                "step_no": 0,
                "role": "user",
                "content": "I love horror stories, are you familiar with them?",
            },
        }
    ),
    MessagePydantic(
        **{
            "id": "abbc7961-45dc-4973-aaf4-a6224ed35d37",
            "header": {
                "sender": "Sara",
                "recipient": "User",
                "timestamp": 1751308167.3488164,
                "order": 1,
            },
            "body": {
                "type": "chat.reply",
                "dialogue_id": "00000000-0000-0000-0000-000000000000",
                "round_no": 0,
                "step_no": 1,
                "chunk_no": 10,
                "chunk": "<eom>",
                "role": "assistant",
                "content": "Yes, I am familiar with horror stories. They are a fascinating genre that can evoke strong emotions and create a sense of suspense and fear. Do you have any specific horror stories in mind that you would like to discuss?",
            },
        }
    ),
]
from gai.lib.constants import DEFAULT_GUID
from gai.messages import Dialogue
dialogue = Dialogue(messages=messages)
recap = dialogue.extract_recap()

#---

from gai.asm import AgenticStateMachine

with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> CHAT
    CHAT--> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                    "recap": {"type": "getter", "dependency": "get_recap"},
                }
            },
            "CHAT": {
                "module_path": "gai.asm.agents.tool_use_agent",
                "class_name": "AnthropicChatState",
                "title": "CHAT",
                "input_data": {
                    "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                    "recap": {
                        "type": "state_bag",
                        "dependency": "recap",
                    },
                },
                "output_data": ["streamer", "get_assistant_message"],
            },
            "FINAL": {
                "output_data": ["monologue"],
            },
        },
        get_llm_config=lambda state: llm_config,
        get_recap=lambda state: recap,
    )
    fsm.restart()

## Step 2: INIT --> CHAT

fsm.user_message = "Tell me a one sentence story."
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    if isinstance(chunk, str):
        print(chunk, end="", flush=True)
print("\n\n")


The old lighthouse keeper discovered that old lighthouse keeper discovered that the light he'd been t the light he'd been tending for forty yearsending for forty years wasn't guiding ships to safety, wasn't guiding ships to safety, but rather calling ancient sea creatures home from but rather calling ancient sea creatures home from the deepest trenches of the ocean. the deepest trenches of the ocean.









### b) Tool Call Flow

In [4]:
from gai.asm import AgenticStateMachine
from gai.mcp.client import McpAggregatedClient

with AgenticStateMachine.StateMachineBuilder(
    """
    INIT --> CHAT
    CHAT--> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                    "mcp_client": {"type": "getter", "dependency": "get_mcp_client"},
                }
            },
            "CHAT": {
                "module_path": "gai.asm.agents.tool_use_agent",
                "class_name": "AnthropicChatState",
                "title": "CHAT",
                "input_data": {
                    "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                    "mcp_client": {"type": "state_bag", "dependency": "mcp_client"},
                },
                "output_data": ["streamer", "get_assistant_message"],
            },
            "FINAL": {
                "output_data": ["monologue"],
            },
        },
        get_llm_config=lambda state: llm_config,
        get_mcp_client=lambda state: McpAggregatedClient(["mcp-time"]),
    )
    fsm.restart()

## Step 2: INIT --> CHAT

fsm.user_message = "What time is it in Singapore?"
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    if (isinstance(chunk,str)):
        print(chunk, end='', flush=True)
print("\n\n")

## Step 4: Print the state history
for message in fsm.state_history[1]["output"]["monologue"].list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

I'll check the current time in Singapore for you. the current time in Singapore for you.



1754270316.4746602 User > 
            Your name is Assistant within the context of this conversation and you will always respond as such.
            Do not refer to yourself as an AI or a bot or confuse your name with other agents.
           
            You may respond to my following message using the context you have learnt.
            What time is it in Singapore?
            
            You may ask me for more information if you need to clarify my request but ask just enough to get the information you need to get started.
            If you need to ask for more information, please use the "user_input" tool to get the information from me.
            Never call "user_input" without providing a description of what you need from me.
            Do not be vague and expect an input from me, be specific about what you want.
            
1754270326.7707736 Assistant > [{'citations': None, 'te